# Project IMDb-Scrape
NB: The notebook is intended for accademic purposes only!

A simple script to scrap information about the top 250 movies from IMDb website!

In [ ]:
# requirements

! pip install beautifulsoup4 \
    requests \
    minio 

In [70]:
# configs comes here
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0"
CACHE_DIR = ".cache"

IMDB_PAGE_LINK = "https://www.imdb.com/chart/top/"

# db configs
WRITE_TO_DB = True
SQLITE_DB_PATH = "IMDb.db"

# upload images to minio
IMAGES_TO_MINIO = False
MINIO_HOST = "localhost"
MINIO_PORT = 9000
MINIO_USER = "minioadmin"
MINIO_PASS = "minioadmin"
MINIO_BUCKET = "movie-posters"


MOVIE_DATA = {
    'obj_id': '',
    'title': '',
    'image_url': '',
    'year': '',
    'certificate': '',
    'duration': '',
    'rating': '',
    'popularity_score': '',
    'num_of_user_reviews': '',
    'num_of_critic_reviews': '',
    'meta_critic_score': '',
    'genres': '',
    'director': '',
    'writers': '',
    'stars': '',
    'award_information': '',
    'description': ""
}



### Setting up the databse and minio functionalities

In [53]:
import sqlite3

class DatabaseHandler: 
    def __init__(self, db_file, table_name:str = "movies", create_tables_if_not_exist=True):
        self.db_file = db_file
        self.table_name = table_name
        if create_tables_if_not_exist:
            self.create_table()

    def _do_changes(self, sql:str, conn = None, params=()):
        _close_connection = False
        if not conn:
            _close_connection = True
            conn = sqlite3.connect(self.db_file)
        try:
            c = conn.cursor()
            c.execute(sql, params)
            conn.commit()
            return True
        except sqlite3.Error as e:
            print("[DB ERROR] :", e)
            return False
        finally:
            if conn and _close_connection:
                conn.close()

    def get_connection(self):
        return sqlite3.connect(self.db_file)

    def create_table(self, conn = None):
        create_users_table_sql = f'''
        CREATE TABLE IF NOT EXISTS {self.table_name} (
            id integer PRIMARY KEY,
            obj_id text,
            title text ,
            image_url text ,
            year text ,
            certificate text ,
            duration text ,
            rating text ,
            popularity_score text ,
            num_of_user_reviews text ,
            num_of_critic_reviews text ,
            meta_critic_score text ,
            genres text ,
            director text ,
            writers text ,
            stars text ,
            award_information text,
            description text);
        '''
        return self._do_changes(create_users_table_sql, conn=conn)
    
    def add_data(self, conn, **kwagrs):
        sql = f''' INSERT INTO {self.table_name} (obj_id, title, image_url, year, certificate, duration, 
            rating, popularity_score, num_of_user_reviews, num_of_critic_reviews, meta_critic_score, 
            genres, director, writers, stars, award_information, description) 
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
        return self._do_changes(sql, conn, params=(
            kwagrs.get("obj_id"),  kwagrs.get("title"),  kwagrs.get("image_url"),  kwagrs.get("year"),
            kwagrs.get("certificate"),  kwagrs.get("duration"),
            kwagrs.get("rating"), kwagrs.get("popularity_score"), kwagrs.get("num_of_user_reviews"),
            kwagrs.get("num_of_critic_reviews"),  kwagrs.get("meta_critic_score"),
            kwagrs.get("genres"),  kwagrs.get("director"),  kwagrs.get("writers"),  kwagrs.get("stars"),
            kwagrs.get("award_information"),  kwagrs.get("description") 
        ))
    
    def select_all_data(self, table_name:str = "movies"):
        conn = sqlite3.connect(self.db_file)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM {table_name}")
        rows = cur.fetchall()
        for row in rows:
            print(row)



In [68]:
import tempfile
import requests
from minio import Minio

class MinioManager:
    client = Minio( f"{MINIO_HOST}:{MINIO_PORT}", 
        access_key=MINIO_USER,
        secret_key=MINIO_PASS,
        secure=False
    )

    @classmethod
    def setup(cls, bucket_name:str = MINIO_BUCKET):
        # Make the bucket if it doesn't exist.
        found = cls.client.bucket_exists(bucket_name)
        if not found:
            cls.client.make_bucket(bucket_name)
            print("[MINIO] Created bucket", bucket_name)
        else:
            print("[MINIO] Bucket", bucket_name, "already exists")


    @classmethod
    def get_and_put_data_to_minio(cls, url:str, minio_path:str, minio_bucket:str = MINIO_BUCKET):
        # download image
        with tempfile.NamedTemporaryFile("wb", suffix=f".{url.split('?')[0].split('.')[-1]}") as file:
            with requests.get(url, headers={'User-Agent': USER_AGENT}, stream=True) as resp:
                resp.raise_for_status()
                for chunk in resp.iter_content(chunk_size=8192):
                    file.write(chunk)
            
            # save image
            cls.client.fput_object(
                minio_bucket, f"{minio_path}/{url.split('?')[0].split('/')[-1]}", file.name,
            )
        


# MinioManager.get_and_put_data_to_minio("https://m.media-amazon.com/images/M/MV5BZTdmNjY5ODYtYTJlMi00ZWY0LTg0YWQtZjY5NWM2ZGYzOTA2XkEyXkFqcGdeQXVyMTA2ODkwNzM5._V1_.jpg", "test")

### Getting into the Scraping

In [55]:
# getting total movie list and their urls
import os
from typing import List

import requests
from bs4 import BeautifulSoup

def _get_root_url(url:str):
    return "/".join(url.split("/")[:3])

def _get_resolved_url(url:str, current_url:str) -> str:
    if url[0] == "/" :
        url = _get_root_url(current_url) + url
    elif url[:2] == "./" :
        url = current_url.rstrip("/") + url.lstrip(".")
    return url

def _get_page(url:str, cache:str="", cache_dir:str = CACHE_DIR) -> str:
    if cache: cache = os.path.join(CACHE_DIR, cache)

    # return cache
    if cache and os.path.exists(cache):
        with open(cache, 'r') as f:
            return f.read()
        
    headers = {
        'User-Agent': USER_AGENT
    }
    res = requests.get(url, headers=headers)
    if res.status_code > 299 or res.status_code < 200:
        raise Exception("didn't get the site", res.status_code)

    # do caching
    if cache:
        os.makedirs(CACHE_DIR, exist_ok=True)
        with open(cache, 'w') as f:
            f.write(res.text)
    return res.text

# and we are going to scrape
def get_movie_page_links(url:str = IMDB_PAGE_LINK) -> List[str]:
    text = _get_page(url, "main-page.html")

    soup = BeautifulSoup(text, 'html.parser')

    # container div
    inner_soup = soup.find("ul", class_="ipc-metadata-list")
    # and the list
    a_tags = inner_soup.find_all("a", class_="ipc-lockup-overlay ipc-focusable")

    urls = []
    for a_tag in a_tags:
        urls.append(_get_resolved_url(a_tag["href"], url))


    # that's enough for now
    return urls

# get_movie_page_links()

In [56]:
# getting movie wise information

def _get_image_url_from_poster_page(page_url:str) -> str:
    img_page = _get_page(page_url, "-".join(page_url.split("/")[-2:]))
    divs = BeautifulSoup(img_page).find("div", class_="media-viewer").find_all("div")
    for div in divs:
        if div and div.find("img"):
            return _get_resolved_url(div.find("img")["src"], page_url)
    raise Exception("Image not found !")

def get_data_from_movie_page(page_url:str, default_data:dict=MOVIE_DATA) -> dict:
    data = {**default_data}
    data["obj_id"] = page_url.split("/")[-2]
 
    try:
        page = _get_page(page_url , "-".join(page_url.split("/")[-2:]))
    except Exception as exp:
        print("[PAGE ERROR] ERROR LOADING PAGE!", exp, page_url)
        return data
 
    soup = BeautifulSoup(page)

    try:
        portion_of_interests = soup.find("main").find("section").find("section").find_all("div", class_="ipc-page-content-container")
    except Exception as exp:
        print("[PAGE ERROR] error getting INNER SECTION! :", exp, page_url)
        return data
    
    for portion in portion_of_interests:
        section = portion.find("section", class_="ipc-page-background")
        if section and section.find(class_="hero__primary-text"):           
            # title     
            title_sec = section.find('span', class_="hero__primary-text")
            data["title"] = title_sec.string

            # getting the image
            try:
                img_a_tag = section.find("div", class_="ipc-poster--baseAlt").find("a")
                data["image_url"] = _get_image_url_from_poster_page(_get_resolved_url(img_a_tag.get("href"), page_url))
            except Exception as exp:
                print("[PAGE ERROR] error getting IMAGE :", exp, page_url)
            
            # hunt for year certificate and duration
            try:
                li = title_sec.parent.parent.find("ul", class_="ipc-inline-list").find_all("li")
                data["year"] = li[0].string
                data["certificate"] = li[1].string
                data["duration"] = li[2].string
            except Exception as exp:
                print("[PAGE ERROR] error getting YEAR/CERT/DURATION :", exp, page_url)
    
            # get the rating 
            try:
                data["rating"] = section.find("div", attrs={"data-testid": "hero-rating-bar__aggregate-rating__score"}).find("span").string
                data["popularity_score"] = section.find("div", attrs={"data-testid": "hero-rating-bar__popularity__score"}).string
            except Exception as exp:
                print("[PAGE ERROR] error getting IMDB RATING :", exp, page_url)

            # getting number of reviewers
            try:
                review_content_secs = section.find("ul", attrs={"data-testid": "reviewContent-all-reviews"}).find_all("li")

                data["num_of_user_reviews"] = review_content_secs[0].find("span", class_="score").string
                data["num_of_critic_reviews"] = review_content_secs[1].find("span", class_="score").string
                data["meta_critic_score"] = review_content_secs[2].find("span", class_="score").string
            except Exception as exp:
                print("[PAGE ERROR] error getting USER REVIEW COUNT :", exp, page_url)

            # getting descriptive part
            try:
                data["genres"] = section.find("div", attrs={"data-testid": "genres"}).find("a").string

                people_sec_ul = section.find("ul", class_="ipc-metadata-list--baseAlt").find_all("li", attrs={"data-testid": "title-pc-principal-credit"})
                data["director"] = people_sec_ul[0].find("div").string
                data["writers"] = ",".join([li.string for li in people_sec_ul[1].find("ul").find_all("li")])
                data["stars"] = ",".join([li.string for li in people_sec_ul[2].find("ul").find_all("li")])
            except Exception as exp:
                print("[PAGE ERROR] error getting DESCRIPTIVE PART :", exp, page_url)

            # award iformation
            try:
                awards = soup.find("li", attrs={"data-testid": "award_information"}).children
                data["award_information"] = ",".join([el.string for el in awards if el.string])
            except Exception as exp:
                print("[PAGE ERROR] error getting AVARD INFORMATION! :", exp, page_url)

            # description
            try:
                data["description"] = section.find("span", attrs={"data-testid": "plot-xl"}).string
            except Exception as exp:
                print("[PAGE ERROR] error getting DESCRIPTION! :", exp, page_url)
    
    return data

# # and the testing part
# _get_movie_page_links(IMDB_PAGE_LINK)
# get_data_from_movie_page('https://www.imdb.com/title/tt0111161/?ref_=chttp_i_1')

In [ ]:
# and the data collection part
import csv
from multiprocessing import Pool, Value

# setting up a minio 
file = open("out.csv", 'w')
csv_writer = csv.writer(file)
csv_writer.writerow(MOVIE_DATA.keys())
file.flush()

# setup db 
db = None
connection = None
if WRITE_TO_DB:
    db = DatabaseHandler(SQLITE_DB_PATH)
    connection = db.get_connection()

if IMAGES_TO_MINIO:
    MinioManager.setup()    

# counter
counter = Value('i', 0)

def process_one_movie(url):
    dat = get_data_from_movie_page(url)
    csv_writer.writerow(dat.values())
    file.flush()

    if WRITE_TO_DB:
        db.add_data(connection, **dat)

    if IMAGES_TO_MINIO:
        try:
            MinioManager.get_and_put_data_to_minio(dat["image_url"], dat["obj_id"])
        except Exception as exp:
            print("[MINIO ERROR] Error downloading / uploading poster image to minio :", exp)
    with counter.get_lock():
        counter.value += 1
        print("\rPROCESSED : {}/{}".format(counter.value, 250), end="")

with Pool(10) as pool:
    print("Starting...", end="")
    pool.map(process_one_movie, get_movie_page_links(IMDB_PAGE_LINK))


# garbage collection
if connection:
    connection.close()
file.close()